In [1]:
%load_ext autoreload
%autoreload 2

import inspect
from fastai.text import * 
import fastai; fastai.__version__

'1.0.39.dev0'

## current

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)

df = pd.read_csv(path/'texts.csv')

# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [3]:
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=4)

In [4]:
learn = language_model_learner(data_lm, drop_mult=0.5)
learn.save_encoder('ft_enc')

RuntimeError: CUDA error: out of memory

In [ ]:
learn = text_classifier_learner(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')

`get_preds` works for `DatasetType.Valid`

In [ ]:
pred_val, y_val=learn.get_preds(DatasetType.Valid, ordered=True)

pred_val.shape, y_val.shape

In [ ]:
assert np.alltrue(learn.data.valid_ds.y.items==y_val.numpy())

`get_preds` works for `DatasetType.Train` for `ordered=False`

In [ ]:
pred_trn, y_trn=learn.get_preds(DatasetType.Train, ordered=False)
pred_trn.shape, y_trn.shape, len(learn.data.train_ds)

`get_preds` does not work for `DatasetType.Train` for `ordered=True`

In [ ]:
pred_trn, y_trn=learn.get_preds(DatasetType.Train, ordered=True)
pred_trn.shape, y_trn.shape, len(learn.data.train_ds)

## This change generate the desired result, but is there a real solution?

Code change in `TextClasDataBunch.create`

In [11]:
print(inspect.getsource(TextClasDataBunch.create))

    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs=64, pad_idx=1, pad_first=True,
               no_check:bool=False, **kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification."
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        collate_fn = partial(pad_collate, pad_idx=pad_idx, pad_first=pad_first)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        # I removed >>>
        # train_dl = DataLoader(datasets[0], batch_size=bs//2, sampler=train_sampler, drop_last=True, **kwargs)
        # <<< I removed
        # I added >>> 
        train_dl = DataLoader(datasets[0], batch_size=bs//2, sampler=train_sampler, drop_last=False, **kwargs)
        # <<< I added >>> 
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=

Code change in `SortishSampler`

In [12]:
print(inspect.getsource(SortishSampler))

class SortishSampler(Sampler):
    "Go through the text data by order of length with a bit of randomness."

    def __init__(self, data_source:NPArrayList, key:KeyFunc, bs:int):
        self.data_source,self.key,self.bs = data_source,key,bs

    def __len__(self) -> int: return len(self.data_source)

    def __iter__(self):
        
        # I added >>> 
        np.random.seed(42)
        # <<< I added
        idxs = np.random.permutation(len(self.data_source))
        sz = self.bs*50
        ck_idx = [idxs[i:i+sz] for i in range(0, len(idxs), sz)]
        sort_idx = np.concatenate([sorted(s, key=self.key, reverse=True) for s in ck_idx])
        sz = self.bs
        ck_idx = [sort_idx[i:i+sz] for i in range(0, len(sort_idx), sz)]
        max_ck = np.argmax([self.key(ck[0]) for ck in ck_idx])  # find the chunk with the largest key,
        ck_idx[0],ck_idx[max_ck] = ck_idx[max_ck],ck_idx[0]     # then make sure it goes first.
        sort_idx = np.concatenate(np.random.permutation(ck_i

In [13]:
path = untar_data(URLs.IMDB_SAMPLE)

df = pd.read_csv(path/'texts.csv')

# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [14]:
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [15]:
learn = language_model_learner(data_lm, drop_mult=0.5)
learn.save_encoder('ft_enc')

In [16]:
learn = text_classifier_learner(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [17]:
pred_val, y_val=learn.get_preds(DatasetType.Valid, ordered=True)

pred_val.shape, y_val.shape

(torch.Size([201, 2]), torch.Size([201]))

In [18]:
assert np.alltrue(learn.data.valid_ds.y.items==y_val.numpy())

In [20]:
pred_trn, y_trn=learn.get_preds(DatasetType.Train, ordered=False)
pred_trn.shape, y_trn.shape, len(learn.data.train_ds)

(torch.Size([799, 2]), torch.Size([799]), 799)

In [21]:
pred_trn, y_trn=learn.get_preds(DatasetType.Train, ordered=True)
pred_trn.shape, y_trn.shape, len(learn.data.train_ds)

(torch.Size([799, 2]), torch.Size([799]), 799)

In [23]:
assert np.alltrue(learn.data.train_ds.y.items==y_trn.numpy())

## Solution

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)

df = pd.read_csv(path/'texts.csv')

# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')

In [3]:
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=4)

In [4]:
learn = language_model_learner(data_lm, drop_mult=0.5)
learn.save_encoder('ft_enc')

In [5]:
learn = text_classifier_learner(data_clas, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [11]:
pred_fix, y_fix=learn.get_preds(DatasetType.Fix, ordered=True)
pred_fix.shape, y_fix.shape, len(learn.data.train_ds)

(torch.Size([799, 2]), torch.Size([799]), 799)

In [13]:
assert np.alltrue(learn.data.train_ds.y.items==y_fix.numpy())

In [14]:
pred_fix, y_fix=learn.get_preds(DatasetType.Fix)
pred_fix.shape, y_fix.shape, len(learn.data.train_ds)

(torch.Size([799, 2]), torch.Size([799]), 799)

In [15]:
assert np.alltrue(learn.data.train_ds.y.items==y_fix.numpy())

AssertionError: 